## Guide

#### 1. Excel File Format:

File Type: Ensure the file you want to process is in .xlsx format.

Columns:

URL: This column should contain URLs with embedded 19-digit post IDs. The code in this notebook will extract these post IDs to derive timestamps.

Sample Excel File Structure:

URL	                                                            
https://example.com/post/1234567890123456789

#### 2. Using the Code in This Notebook:

Navigate to the cell containing the line:

input_file = "trial.xlsx"


Modify this line to point to the path of your Excel file. For example:

input_file = "path_to_your_file.xlsx"


#### 3. Running the Code:

Execute the cell containing the code by selecting it and pressing Shift + Enter.

#### 4. Checking the Results:

After the cell execution completes, open the Excel file you provided.

You should now see the 'Timestamp UTC' and 'Timestamp IST' columns populated with the extracted timestamps in human-readable date formats for each URL in the 'URL' column.

#### 5. Notes:

The code assumes that the URLs in the 'URL' column contain a 19-digit post ID from which a Unix timestamp can be extracted.

The extracted Unix timestamp is then converted to human-readable date formats in both UTC and IST.

The updated data is saved back to the original Excel file.

# TimeStamp Extraction Code

In [ ]:
import re
from datetime import datetime, timezone, timedelta
import pandas as pd

def get_post_id(url):
    if not isinstance(url, str):
        return None
    match = re.search(r'([0-9]{19})', url)
    if match:
        return match.group(1)
    return None

def extract_unix_timestamp(post_id):
    as_binary = bin(int(post_id))[2:]
    first41_chars = as_binary[:41]
    timestamp = int(first41_chars, 2)
    return timestamp

def unix_timestamp_to_human_date(timestamp, to_ist):
    timestamp = timestamp / 1000
    utc_time = datetime.fromtimestamp(timestamp, tz=timezone.utc)
    if to_ist:
        ist_time = utc_time + timedelta(hours=5, minutes=30)
        return ist_time.strftime('%a, %d %b %Y %I:%M:%S %p IST')
    else:
        return utc_time.strftime('%a, %d %b %Y %I:%M:%S %p UTC')

def process_xlsx_file(input_file):
    df = pd.read_excel(input_file)

    if 'Timestamp UTC' not in df.columns:
        df['Timestamp UTC'] = None
    if 'Timestamp IST' not in df.columns:
        df['Timestamp IST'] = None

    for idx, row in df.iterrows():
        if pd.notna(row['URL']):
            post_id = get_post_id(row['URL'])
            if post_id:
                unix_timestamp = extract_unix_timestamp(post_id)
                df.at[idx, 'Timestamp UTC'] = unix_timestamp_to_human_date(unix_timestamp, to_ist=False)
                df.at[idx, 'Timestamp IST'] = unix_timestamp_to_human_date(unix_timestamp, to_ist=True)

    df.to_excel(input_file, index=False)
    return df

input_file = "URL.xlsx"
result_df = process_xlsx_file(input_file)
display(result_df)


,URL,Timestamp UTC,Timestamp IST
0,https://www.linkedin.com/posts/pankhuriguptama...,"Fri, 14 Jul 2023 04:55:01 AM UTC","Fri, 14 Jul 2023 10:25:01 AM IST"
1,https://www.linkedin.com/posts/pankhuriguptama...,"Thu, 13 Jul 2023 05:36:07 AM UTC","Thu, 13 Jul 2023 11:06:07 AM IST"
2,https://www.linkedin.com/posts/pankhuriguptama...,"Thu, 29 Jun 2023 06:17:31 AM UTC","Thu, 29 Jun 2023 11:47:31 AM IST"
3,https://www.linkedin.com/posts/pankhuriguptama...,"Thu, 22 Jun 2023 05:30:15 AM UTC","Thu, 22 Jun 2023 11:00:15 AM IST"
4,https://www.linkedin.com/posts/pankhuriguptama...,"Tue, 20 Jun 2023 03:52:10 AM UTC","Tue, 20 Jun 2023 09:22:10 AM IST"
...,...,...,...
61,https://www.linkedin.com/posts/writeraainachop...,"Mon, 10 Jul 2023 07:14:05 AM UTC","Mon, 10 Jul 2023 12:44:05 PM IST"
62,https://www.linkedin.com/posts/writeraainachop...,"Sun, 09 Jul 2023 06:49:56 AM UTC","Sun, 09 Jul 2023 12:19:56 PM IST"
63,https://www.linkedin.com/posts/writeraainachop...,"Sat, 08 Jul 2023 08:52:43 AM UTC","Sat, 08 Jul 2023 02:22:43 PM IST"
64,https://www.linkedin.com/posts/writeraainachop...,"Fri, 07 Jul 2023 03:44:14 AM UTC","Fri, 07 Jul 2023 09:14:14 AM IST"
